In [1]:
import time
import pygame
import pygame.midi
import numpy as np
from music21 import *
from PIL import Image
from pythonosc import udp_client

pygame 2.6.1 (SDL 2.28.4, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


/Users/adrian/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Loading data

In [2]:
lc_names = np.load('lc_names.npy')
fluxes = np.load('fluxes.npy', allow_pickle=True)
flux_errors = np.load('flux_errors.npy', allow_pickle=True)
times = np.load('times.npy', allow_pickle=True)
problems = np.load('problems.npy', allow_pickle=True)
dims = np.load('dims.npy')

## Load any MIDI file

In [3]:
file = 'MIDI/MozartSky.mid'

In [4]:
midi_score = converter.parse(file) #creates a Music 21 score

In [5]:
midi_score

<music21.stream.Score 0x10f7587f0>

In [6]:
notes = []
midi_notes = []

# Iterates through all notes and chords in the score
for element in midi_score.flat.notes:
    if isinstance(element, note.Note): # Single note pitch
        notes.append(str(element.pitch))  
        midi_notes.append({'pitch': element.pitch.midi, 'duration': element.quarterLength})

    elif isinstance(element, chord.Chord): # For chords, appends all pitches as a list
        notes.append('.'.join(str(n) for n in element.pitches))
        for pitch in element.pitches:
            midi_notes.append({'pitch': pitch.midi, 'duration': element.quarterLength})
        
# Prints the extracted notes
print(notes)


['C#4', 'D5', 'C#6', 'E5', 'B-4', 'C4', 'E6', 'D4', 'C4', 'A3', 'D4', 'C4', 'B-3', 'D4', 'A3', 'D5', 'B-4', 'B-3', 'E-2', 'B-4', 'A4', 'F3', 'B-3', 'E-3', 'B-4', 'G4', 'D3', 'B-3', 'E-2', 'E-3', 'B-4', 'E-5', 'B-3', 'E-3', 'B-4', 'B-4', 'B-5', 'B-3', 'G4', 'E-3', 'B-4', 'D5', 'C5', 'B-3', 'C3', 'B-4', 'D4.C5', 'E-3', 'B-3', 'B-4', 'E-3', 'E-3', 'B-4', 'F4', 'B-4', 'E-6', 'F4', 'E-3', 'B-4', 'F4', 'A4', 'E-3', 'B-2', 'B-5.E-6.F6', 'B-4', 'B-3.F4', 'F5', 'B-4', 'A3', 'B-4', 'E-5.B-5', 'B-2.B-3', 'F3', 'C5', 'A4', 'F3', 'A5', 'B-3', 'B-4', 'E-2', 'F6', 'B-3', 'B-4', 'C5', 'E-3', 'B-4', 'E-3', 'F6', 'E-5', 'B-4', 'B-3', 'E-3', 'D6', 'F6', 'D6', 'E-3.C5', 'A3.D4.E-5', 'B-4', 'B-3', 'A4', 'F6', 'C6', 'E-5', 'C6', 'B-3', 'E-5', 'C6', 'B-3', 'C5', 'A4', 'E-6', 'D5', 'A4', 'B-4', 'E-3', 'B-5.D6', 'B-4', 'B-3', 'B-3.C5', 'D4', 'E-3', 'A5.C6', 'B-4', 'E-2', 'E-3', 'G4', 'E-3', 'G4', 'F6', 'F#2', 'B-5', 'B-4', 'C6.E-6', 'B-5', 'E-3.G4.E-5', 'B-3', 'E-3.G4.E-5', 'E-6', 'F3', 'E-3', 'B-2', 'F3', 'E-

In [7]:
notes[10] #Example of chord (from the score) access

'D4'

## Loading the periods found in the light curves

In [8]:
best_periods = np.load('OMC_Analysis/Results/best_periods.npy')
best_powers = np.load('OMC_Analysis/Results/best_powers.npy')
poly_fits = np.load('OMC_Analysis/Results/poly_fits.npy', allow_pickle=True)

In [9]:
best_periods[0]*100 # Testing to scale the values to audible range

779.3879387938795

In [10]:
ansi_chords = (np.ceil(100*best_periods)).astype(int)
lc_notes = [[] for _ in ansi_chords]
for i in range(len(ansi_chords)):
    lc_notes[i].append(pygame.midi.midi_to_ansi_note(ansi_chords[i]))

In [11]:
print(ansi_chords) #The array of periods converted into ANSI codes

[780  56 118 ...  56 272  74]


In [12]:
lc_notes #The periods converted into MIDI notes

[['C64'],
 ['G#3'],
 ['A#8'],
 ['F21'],
 ['F3'],
 ['D#3'],
 ['D#3'],
 ['D#17'],
 ['G#6'],
 ['A15'],
 ['D#6'],
 ['D6'],
 ['A#4'],
 ['B34'],
 ['C#4'],
 ['D#4'],
 ['G4'],
 ['C8'],
 ['A#3'],
 ['F#23'],
 ['D78'],
 ['A#27'],
 ['G14'],
 ['G3'],
 ['A#3'],
 ['D3'],
 ['A77'],
 ['C14'],
 ['D4'],
 ['B56'],
 ['G#5'],
 ['A80'],
 ['C50'],
 ['C21'],
 ['C#4'],
 ['C#46'],
 ['F12'],
 ['F3'],
 ['G5'],
 ['G#6'],
 ['C#56'],
 ['G#6'],
 ['C33'],
 ['F#3'],
 ['G11'],
 ['E4'],
 ['F#4'],
 ['B3'],
 ['B19'],
 ['A#17'],
 ['D6'],
 ['D55'],
 ['C#25'],
 ['A9'],
 ['E14'],
 ['A24'],
 ['G3'],
 ['F75'],
 ['A5'],
 ['A#6'],
 ['A13'],
 ['G20'],
 ['D#24'],
 ['A36'],
 ['B4'],
 ['A#5'],
 ['E44'],
 ['F28'],
 ['C#4'],
 ['F#27'],
 ['D#3'],
 ['F4'],
 ['D#3'],
 ['E23'],
 ['C#4'],
 ['C9'],
 ['B13'],
 ['F5'],
 ['D#10'],
 ['A#6'],
 ['F4'],
 ['C#6'],
 ['C12'],
 ['F7'],
 ['A#4'],
 ['D#3'],
 ['G18'],
 ['G4'],
 ['B54'],
 ['D#4'],
 ['F3'],
 ['B4'],
 ['B3'],
 ['B11'],
 ['C35'],
 ['C#10'],
 ['F#3'],
 ['D6'],
 ['G21'],
 ['F#14'],
 ['D#7'],
 ['D

In [13]:
lc_notes[14][0] #Example of note (from the periodograms)

'C#4'

In [14]:
notes[0] #Example of note (from the MIDI score)

'C#4'

In [15]:
lc_notes[14][0] == notes[0] #Boolean test to prove the condition for the Pitch Class Set Theory (Matching Root chord notes)

True

In [16]:
chord1 = chord.Chord(lc_notes[14][0]) #Conversion into Music 21 chord object to support polyphonic chords
chord2 = chord.Chord(notes[0])

In [17]:
chord1.root () == chord2.root ()

True

## Generating a score with the periods

In [18]:
print(min(ansi_chords), max(ansi_chords))

50 981


In [19]:
ansi_chords2 = [50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

In [20]:
def normalize_to_midi(values, original_min=50, original_max=981, midi_min=0, midi_max=127):
    # Normalizes to 0-1
    normalized = (values - original_min) / (original_max - original_min)
    # Scales to MIDI range
    midi_scaled = normalized * (midi_max - midi_min) + midi_min
    # Rounds and convert to integers within MIDI range
    midi_scaled = np.clip(np.round(midi_scaled), midi_min, midi_max).astype(int)
    return midi_scaled

In [21]:
midi_values = normalize_to_midi(ansi_chords)
print(midi_values)

[100   1   9 ...   1  30   3]


In [22]:
period_score = stream.Stream()

# Convert each note name to a Note object and add to the score
for n in midi_values:
    new_note = note.Note(n)
    period_score.append(new_note)

# Show the score (opens in music notation viewer or outputs MusicXML)
period_score.show('musicXML')

## Launching Multimodal Exploration

In [23]:
pygame.midi.init()
for i in range(pygame.midi.get_count()):
    info = pygame.midi.get_device_info(i)
    (interf, name, is_input, is_output, opened) = info
    print(f"ID: {i}, Name: {name.decode()}, Input: {is_input}, Output: {is_output}")
pygame.midi.quit() #Returns the list of available MIDI devices

ID: 0, Name: GarageBand Virtual Out, Input: 1, Output: 0
ID: 1, Name: GarageBand Virtual In, Input: 0, Output: 1


In [ ]:
# Initializes pygame
pygame.init()
# Initializes midi
pygame.midi.init()

device_id = 1  # replace with your actual output device ID
player = pygame.midi.Output(device_id)

# Define MIDI channels and instruments if needed
channel = 0
program = 0  # Acoustic Grand Piano
player.set_instrument(program, channel)
count = 0
path = 'OMC_Analysis/Fitted/'

#Play note by note
for i in range(len(notes)):
    try:
        print("Pitch:", midi_notes[i]['pitch'])
        chord1 = chord.Chord(notes[i])
        lc_found = 0
        for j in range(len(lc_notes)):
            chord2 = chord.Chord(lc_notes[j][0])
            if (chord1.root () == chord2.root ()):
                print("Light curve:", lc_names[j])
                lc_found = 1                
                name = lc_names[j][24:34]
                img_png = path + name + '.png'

                #Saving the image
                image = Image.open(img_png)
                image.save('OMC_Analysis/REC/LC'f'{count}.png')
                image.close()

                count += 1
                break
        if lc_found == 0:
            index = int(np.random.uniform(low=0, high=len(lc_notes)))
            lc_name = lc_names[index][24:34]
            print("Light curve not found. Displaying a random light curve", lc_name)
            img_png = path + lc_name + '.png'
            image = Image.open(img_png)
            image.save('OMC_Analysis/REC/LC'f'{count}.png')
            image.close() 
            count += 1
        time.sleep(midi_notes[i]['duration'] * 0.5)  # Scale duration as needed
    
    except:
        print("Process error")
        lc_name = lc_names[23][24:34]
        img_png = path + lc_name + '.png'
        image = Image.open(img_png)
        image.save('OMC_Analysis/REC/LC'f'{count}.png')
        image.close() 
        count += 1
        
print("Lightcurves found", count)
# Cleanup
player.close()
pygame.midi.quit()
pygame.quit()


Pitch: 61
Light curve: OMC Light curve. OMCID: 2788000021, type 0004
Pitch: 74
Light curve: OMC Light curve. OMCID: 3348000020, type 0004
Pitch: 85
Light curve: OMC Light curve. OMCID: 4689000031, type 0004
Pitch: 76
Light curve: OMC Light curve. OMCID: 9184000037, type 0004
Pitch: 70
Light curve not found. Displaying a random light curve 4246000041
Pitch: 60
Light curve: OMC Light curve. OMCID: 6503000043, type 0004
Pitch: 88
Light curve: OMC Light curve. OMCID: 2407000028, type 0004
Pitch: 62
Light curve: OMC Light curve. OMCID: 4016000079, type 0004
Pitch: 60
Light curve: OMC Light curve. OMCID: 6503000043, type 0004
Pitch: 57
Light curve: OMC Light curve. OMCID: 0705000032, type 0004
Pitch: 62
Light curve: OMC Light curve. OMCID: 4016000079, type 0004
Pitch: 60
Light curve: OMC Light curve. OMCID: 6503000043, type 0004
Pitch: 58
Light curve not found. Displaying a random light curve 3961000036
Pitch: 62
Light curve: OMC Light curve. OMCID: 4016000079, type 0004
Process error
Pitch: